In [3]:
from finiteelementanalysis import visualize as viz
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis import solver_demo_helper_functions as solver_demo
from finiteelementanalysis.solver import hyperelastic_solver
import numpy as np
import matplotlib.pyplot as plt

This notebook will show what happens when we take the small deformation case from the Euler beam solution comparison and apply a large load to it.

We will be performing mesh refinement on both D2_nn3_tri and D2_nn6_tri. The mesh will be improved until the percent chang in max deflection between refinements drops below 1%. In other words:

```math
\frac{u^{max}_{i} - u^{max}_{i-1}}{u^{max}_{i}} < 0.01
```

This is accomplished by simply iterating the problem over a loop.

In [11]:
ny =  1 # change this to refine the mesh
L = 30
H = 1
q = 500
E = 1e6
nu = 0.3
# mu = E / (2.0 * (1.0 + nu))
# kappa = E / (3.0 * (1.0 - 2.0 * nu))
mu = E / (2.0 * (1.0 + nu))
#kappa = E / (2.0 * (1.0 - nu))
kappa = E / (3.0 * (1.0 - 2.0 * nu))
material_props = np.array([mu, kappa])

for ele_type in ["D2_nn3_tri", "D2_nn6_tri"]:
    max_disp = 0
    for n in range(5):
        old_max_disp = max_disp
        nx = 5*(n+1)
        coords, connect = pre.generate_rect_mesh_2d(ele_type, 0.0, 0.0, L, H, nx, ny)
        # Identify boundaries
        boundary_nodes, boundary_edges = pre.identify_rect_boundaries(coords, connect, ele_type, 0.0, L, 0.0, H)
        # 1. Fix left boundary: both u_x and u_y = 0.
        fixed_nodes_left = pre.assign_fixed_nodes_rect(boundary_nodes, "left", 0.0, 0.0)
        fixed_nodes_right = pre.assign_fixed_nodes_rect(boundary_nodes, "right", 0.0, 0.0)
        fixed_nodes = np.hstack((fixed_nodes_left, fixed_nodes_right))
        # Assign distributed load on the right boundary
        dload_info = pre.assign_uniform_load_rect(boundary_edges, "top", 0.0, -q)
        # Assign artificial displacement field
        displacement = np.zeros((coords.shape))
        for kk in range(0, coords.shape[0]):
            displacement[kk, 0] = coords[kk, 0] * 0.01

        nr_num_steps = 5
        nr_print = True

        displacements_all, nr_info_all = hyperelastic_solver(material_props, ele_type, coords.T, connect.T, fixed_nodes, dload_info, nr_print, nr_num_steps, nr_tol=1e-9, nr_maxit=30)

        disp_stack = np.stack([d.reshape(-1, 2) for d in displacements_all])
        max_disp = np.max(np.abs(disp_stack))
        fname = f"large-def-{nx}-{ele_type}.gif"
        viz.make_deformation_gif(displacements_all, coords, connect, ele_type, fname)
        if (max_disp - old_max_disp)/max_disp < 0.01:
            break

Step 0, load factor = 0.200
Iteration 1, Correction=1.000000e+00, Residual=5.000000e+01, tolerance=1.000000e-09
Iteration 2, Correction=1.291832e-03, Residual=8.927818e+01, tolerance=1.000000e-09
Iteration 3, Correction=2.123620e-07, Residual=2.166444e-03, tolerance=1.000000e-09
Iteration 4, Correction=1.941456e-15, Residual=1.033484e-10, tolerance=1.000000e-09
Step 1, load factor = 0.400
Iteration 1, Correction=4.997623e-01, Residual=5.000000e+01, tolerance=1.000000e-09
Iteration 2, Correction=1.528476e-03, Residual=8.911865e+01, tolerance=1.000000e-09
Iteration 3, Correction=4.834031e-06, Residual=3.252534e-03, tolerance=1.000000e-09
Iteration 4, Correction=6.801222e-13, Residual=1.021659e-08, tolerance=1.000000e-09
Step 2, load factor = 0.600
Iteration 1, Correction=3.321021e-01, Residual=5.000000e+01, tolerance=1.000000e-09
Iteration 2, Correction=1.838115e-03, Residual=8.791816e+01, tolerance=1.000000e-09
Iteration 3, Correction=6.147828e-06, Residual=5.821032e-03, tolerance=1.000

![alt text](large-def-5-D2_nn3_tri.gif)
![alt text](large-def-10-D2_nn3_tri.gif)
![alt text](large-def-15-D2_nn3_tri.gif)
![alt text](large-def-20-D2_nn3_tri.gif)
![alt text](large-def-25-D2_nn3_tri.gif)
![alt text](large-def-5-D2_nn6_tri.gif)
![alt text](large-def-10-D2_nn6_tri.gif)
![alt text](large-def-15-D2_nn6_tri.gif)

The mesh refinement process iterated on D2_nn3_tri until there were 25 horizontal elements while on D2_nn6_tri it iterated until there were 15 elements. It's very clear that D2_nn6_tri provides a more accurate element, reducing the element by 10 or by 3/5. However, it could be bounded by the fact that even 25 elements on the beam is enough that further increases in mesh size might not be necessary.